# PROYECTO 2023

Descripción proyecto

## Importar librerías

In [ ]:
%pip install numpy --upgrade
%pip install mahotas

In [ ]:
# General
import numpy as np
import math

# Procesamiento de imagenes
import cv2

# Visualizacion
from   tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Extraccion/seleccion de caracteristicas, clasificacion, evaluacion
from   balu3.fx.geo    import fourierdes, hugeo, flusser, gupta,basicgeo # caracteristicas geometricas?
from   balu3.fx.chr    import lbp, haralick, gabor, hog
from   balu3.ft.norm   import minmax
from   balu3.fs.sel    import jfisher,sfs,clean
from   balu3.io.misc   import imageload
from   balu3.cl.basics import ClassifierKNN
from scipy.stats import mode
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from skimage.measure import moments
from skimage.feature import graycomatrix, daisy, graycoprops
from sklearn.decomposition import PCA, FastICA
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, RFE, mutual_info_classif ## fisher_score, 
from mahotas.features import zernike_moments
img = plt.imread("G00/ID004_003.png")
img = img[:, :, 2]
img = img[400-int(400/1.4) : 400+int(400/1.4), 400-int(400/1.4) : 400+int(400/1.4)]
print(img.shape)
descs = hog(img, orientations=HOG_ORIENTATIONS, pixels_per_cell=(
                    img.shape[0], img.shape[1]), cells_per_block=(1, 1))

print(descs.shape)

### Parámetros

In [ ]:
IMG_WIDTH = IMG_HEIGHT = (400 // 1.4) * 2

# LBP
LBP_HDIV = LBP_VDIV = 3
LBP_BINS = 59

# Haralick
HAR_DISTANCE = 3
HAR_SIZE = 24

# Gabor
GAB_ROTATIONS = 8
GAB_DILATATIONS = 8

# HOG
HOG_ORIENTATIONS = 9

# Zernike
ZER_RADIUS = 30
ZER_DEGREE = 8
ZER_SIZE = 25

# GLCM
GLCM_DISTANCES = 1
GLCM_ANGLES = 0 
GLCM_LEVELS = 256

GLCM_SIZE = 1

# Daisy
DAISY_RINGS = 1
DAISY_STEP = 150
DAISY_RADIUS = 50
DAISY_ORIENTATIONS = 8
DAISY_HISTOGRAMS = 5
DAISY_P = math.ceil((IMG_WIDTH - DAISY_RADIUS * 2) / DAISY_STEP) 
DAISY_Q = math.ceil((IMG_HEIGHT - DAISY_RADIUS * 2) / DAISY_STEP)
DAISY_R = (DAISY_RINGS * DAISY_HISTOGRAMS + 1) * DAISY_ORIENTATIONS

# Fetaures
features_per_function = {
    "lbp": LBP_HDIV * LBP_VDIV * LBP_BINS,
    "haralick": HAR_SIZE,
    "gabor": GAB_ROTATIONS * GAB_DILATATIONS + 3,
    "hog": HOG_ORIENTATIONS,
    "zernike": ZER_SIZE,
    "glcm": GLCM_SIZE,
    "daisy": DAISY_P * DAISY_Q * DAISY_R
}
print(features_per_function)

### Funciones auxiliares

In [ ]:
LBP_HDIV = LBP_VDIV = 3
DAISY_RINGS = 1


def extract_features(color_mode, dataset_type, feature_type):
    # Cargar rutas de imágenes y etiquetas según el dataset_type
    # ...
    K = 90   # <= NUMERO DE CLASES

    if dataset_type == 'train':
        fpath = "G00"     # <= DIRERCTORIO DE LA BASE DE DATOS
        N = 12   # <= NUMERO DE IMAGENES POR CLASE
        n = range(12)

    elif dataset_type == 'test':
        fpath = "G01"     # <= DIRERCTORIO DE LA BASE DE DATOS
        N = 4   # <= NUMERO DE IMAGENES POR CLASE
        n = range(12, 16)

    dig_clase = 3     # <= NÚMERO DE DÍGITOS POR CLASE
    dig_img = 3     # <= NÚMERO DE DÍGITOS POR NÚMERO DE IMAGEN
    prefix = "ID"     # <= PREFIJO DEL NOMBRE DEL ARCHIVO DE LA IMAGEN
    imprefix = fpath + '/' + prefix

    # ground truth (clasificacion ideal)
    y = np.zeros((K*N), 'int')
    features = np.zeros((K*N, features_per_function[feature_type]))

    t = 0
    for j in range(K):                  # para cada clase
        for i in tqdm(n):                # para cada imagen de la clase
            # Lectura de la imagen
            clase = j+1
            num_img = i+1
            img = imageload(imprefix, clase, dig_clase,
                            num_img, dig_img, echo='on')
            size = img.shape[0]
            new_size = int(size/1.4)
            img = img[size-new_size: size+new_size,
                      size-new_size: size+new_size]
            y[t] = j+1
            t = t+1

            # Preprocesar las imágenes según el color_mode (gray, red, green, blue)
            if color_mode == 'gray':
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            elif color_mode == 'red':
                img = img[:, :, 0]
            elif color_mode == 'green':
                img = img[:, :, 1]
            elif color_mode == 'blue':
                img = img[:, :, 2]
            else:
                raise ValueError(f"Invalid color mode: {color_mode}")

            # Extraer características según el feature_type
            if feature_type == 'lbp':
                # Aplicar LBP a cada imagen
                features[t:, ] = lbp(img, hdiv=LBP_HDIV, vdiv=LBP_VDIV)

            elif feature_type == 'haralick':
                # Aplicar características de textura Haralick a cada imagen
                # Implementa tu propia función para la extracción de características Haralick
                features[t:, ] = haralick(img, distance=HAR_DISTANCE,)

            elif feature_type == 'gabor':
                # Aplicar filtro de Gabor a cada imagen
                # Implementa tu propia función para la extracción de características Gabor
                features[t:, ] = gabor(
                    img, rotations=GAB_ROTATIONS, dilations=GAB_DILATATIONS)

            elif feature_type == 'hog':
                # Aplicar HOG a cada imagen
                features[t:, ] = hog(img, orientations=HOG_ORIENTATIONS, pixels_per_cell=(
                    img.shape[0], img.shape[1]), cells_per_block=(1, 1))

            elif feature_type == 'zernike':
                # Aplicar momentos de Zernike a cada imagen
                # Implementa tu propia función para la extracción de características Zernike
                features[t:, ] = zernike_moments(img, radius=ZER_RADIUS, degree=ZER_DEGREE)

            elif feature_type == 'glcm':
                img_uint = img.astype(np.uint8)
                # Aplicar GLCM (Matriz de co-ocurrencia de niveles de gris) a cada imagen
                glcm = graycomatrix(img_uint, [GLCM_DISTANCES], [GLCM_ANGLES], levels=GLCM_LEVELS)
                contrast = graycoprops(glcm, 'contrast')
                # Compute desired GLCM properties (e.g., contrast, energy, correlation, etc.)
                features[t:, ] = np.concatenate(contrast)

            elif feature_type == 'daisy':
                # Aplicar Daisy a cada imagen
                features[t:, ] = daisy(img, step=DAISY_STEP, radius=DAISY_RADIUS, rings=DAISY_RINGS,
                                    histograms=DAISY_HISTOGRAMS, orientations=DAISY_ORIENTATIONS).flatten()

            else:
                raise ValueError(f"Invalid feature_type: {feature_type}")

        features = np.array(features)

    # Devolver las características extraídas y las etiquetas
    return features, y


def load_features(color, set, function):
    # !wget https://www.dropbox.com/caracteristicas.npz?dl=0
    # !mv caracteristicas.npz?dl=0 caracteristicas.npz

    loaded_caracteristicas = np.load('caracteristicas.npz')
    y = loaded_caracteristicas['y']
    pass


def select_features(x, y, algorithm):
    if algorithm == 'SFS':
        # SFS: Sequential Feature Selection
        selector = sfs(x, y, n_features=10)
    elif algorithm == 'PCA':
        # PCA: Principal Component Analysis
        selector = PCA(n_components=10)
    # elif algorithm == 'Fisher':
    #     # Fisher Score
    #     selector = SelectKBest(score_func=fisher_score, k=10)
    elif algorithm == 'RFE':
        # Recursive Feature Elimination
        estimator = None  # Ajustar
        selector = RFE(estimator, n_features_to_select=10)
    elif algorithm == 'ICA':
        # ICA: Independent Component Analysis
        selector = FastICA(n_components=10)
    # elif algorithm == 'LFDA':
    #     # LFDA: Local Feature Analysis
    #     selector = LocalFeatureAnalysis(n_features_to_select=10)
    else:
        raise ValueError(f"Invalid algorithm: {algorithm}")

    X_selected = selector.fit_transform(x, y)

    return X_selected.astype(int)


def load_selected_features(algorithm):
    pass


def load_classifier(classifier):
    if classifier == 'knn':
        return KNeighborsClassifier()
    elif classifier == 'lda':
        return LinearDiscriminantAnalysis()
    elif classifier == 'qda':
        return QuadraticDiscriminantAnalysis()
    elif classifier == 'árboles de decisión':
        return DecisionTreeClassifier()
    elif classifier == 'random forest':
        return RandomForestClassifier()
    elif classifier == 'svm lineal':
        return SVC(kernel='linear')
    elif classifier == 'svm rbf':
        return SVC(kernel='rbf')
    elif classifier == 'redes neuronales':
        return MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000)
    else:
        raise ValueError(f"Invalid classifier: {classifier}")


def load_trained_classifier():
    pass


### Cargar imagenes

In [ ]:
!wget https://www.dropbox.com/s/s4opefjionbdbab/G00.zip
!unzip -qq G00.zip

In [ ]:
!wget https://www.dropbox.com/s/zur2wxzcce4qlgf/G01.zip
!unzip -qq G01.zip

## Extracción de caracteristicas

In [ ]:
# Define the extraction models and color modes
extraction_models = ["lbp", "hog", "haralick", "gabor", "glcm", "zernike", "daisy"]
color_modes = ["gray", "red", "green", "blue"]

# Create the "features" folder if it doesn't exist
if not os.path.exists("features"):
    os.makedirs("features")

# Iterate over extraction models and color modes
for model in extraction_models:
    for color_mode in color_modes:
        # Extract features
        (X_train, y_train) = extract_features(color_mode, "train", model)
        (X_test, y_test) = extract_features(color_mode, "test", model)

        # Save features to files
        train_filename = f"features/X_train_{model}_{color_mode}.npy"
        test_filename = f"features/X_test_{model}_{color_mode}.npy"

        np.save(train_filename, X_train)
        np.save(test_filename, X_test)

        print(f"Features saved for {model} - {color_mode}")

### Guardar en un archivo

In [ ]:
np.savez('caracteristicas.npz', y=ytrain)

## Selección de caracteristicas

Descargar caracteristicas desde el archivo y cargarlas

Preprocesamiento de los datos

Seleccionar caracteristicas

In [ ]:
X1 = np.concatenate((X_train_lbp_gray,X_train_lbp_red),axis=1)
X2 = np.concatenate((X_test_lbp_gray ,X_test_lbp_red),axis=1)
sel = select_features(X1,ytrain,algorithm='PCA')
X1_sel = X1[:,sel]
X2_sel = X2[:,sel]

guardar caracteristicas seleccionadas

In [ ]:
np.savez('caracteristicas_seleccionadas.npz', y=y)

## Clasificación

descargar caracteristicas seleccionadas

In [ ]:
# !wget https://www.dropbox.com/caracteristicas_seleccionadas.npz?dl=0
# !mv caracteristicas.npz?dl=0 caracteristicas_seleccionadas.npz

loaded_caracteristicas = np.load('caracteristicas_seleccionadas.npz')

clasificación

### Ensamble de clasificadores

In [ ]:
def majority_voting_ensemble(models, X_test):
    predictions = np.array([model.predict(X_test) for model in models])
    majority_vote = mode(predictions, axis=0)
    return majority_vote.mode.flatten()

## MODELO 1

descripción del modelo

In [ ]:
# Índices de las imágenes del grupo 0
train_indices = np.repeat(np.arange(90), 8)  # Índices para entrenamiento (repetidos 8 veces)
test_indices = np.repeat(np.arange(90), 4)   # Índices para prueba (repetidos 4 veces)

X_train_group0 = X1_sel[train_indices]
y_train_group0 = ytrain[train_indices]
X_test_group0 = X2_sel[test_indices]
y_test_group0 = ytrain[test_indices]

# Dividir el conjunto de entrenamiento en entrenamiento y prueba (Hold Out)
X_train, X_test, y_train, y_test = train_test_split(X_train_group0, y_train_group0, test_size=4, random_state=42)

models = [load_classifier('knn'), load_classifier('lda'), load_classifier('qda'),
          load_classifier('árboles de decisión'), load_classifier('random forest'),
          load_classifier('svm lineal'), load_classifier('svm rbf'),
          load_classifier('redes neuronales')]

best_model = None
best_accuracy = 0.0

# Iterar sobre diferentes modelos y encontrar el que maximice la precisión
for model in models:
    # Entrenar el modelo con el conjunto de entrenamiento
    model.fit(X_train, y_train)

    # Predecir las etiquetas para el conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)

    # Actualizar el mejor modelo si se encuentra uno con una precisión mayor
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

# Reentrenar el mejor modelo con todas las imágenes de grupo 0
best_model.fit(X_train_group0, y_train_group0)

# Entregar el mejor modelo encontrado
entregable_modelo_1 = best_model

## MODELO 2

descripción del modelo

In [ ]:
best_model = None
best_accuracy = 0.0

# Iterar sobre diferentes modelos y encontrar el que maximice la precisión en cross-val
for model in models:
    # Realizar cross-val con 4 folds en las imágenes de grupo 0
    scores = cross_val_score(model, X_train_group0, y_train_group0, cv=4)

    # Calcular la precisión promedio en cross-val
    accuracy = scores.mean()

    # Actualizar el mejor modelo si se encuentra uno con una precisión mayor
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

# Reentrenar el mejor modelo con todas las imágenes de grupo 0
best_model.fit(X_train_group0, y_train_group0)

# Entregar el mejor modelo encontrado
entregable_modelo_2 = best_model